#Intro

In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.

In [10]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.transforms.core import DoFn

In [5]:
!mkdir -p data

In [ ]:
from google.colab import files
uploaded = files.upload()

#Code

##p2

In [21]:
p2 = beam.Pipeline()

# list/array = []
# set = ()
# dictionary = {}

lines = (
            p2
            | beam.Create([
               'Using create transform ',
               'to generate in memory data ',
               'This is 3rd line ',
               'Thanks '])

            | beam.io.WriteToText('data/outCreate1', file_name_suffix='.txt')
          )
p2.run()

In [22]:
!{('head -n 20 /content/data/outCreate1-00000-of-00001.txt')}

Using create transform 
to generate in memory data 
This is 3rd line 
Thanks 


In [19]:
class AddLineNumbersFn(DoFn):
    """Dodaje numer linii do każdego elementu."""
    def __init__(self):
        self.index = 0  # Inicjalizuje licznik indeksów jako 0.

    def process(self, element):
        # Zwiększa licznik o 1 dla każdego elementu.
        self.index += 1
        # Dodaje numer linii do elementu i zwraca w formacie "{numer}. {element}".
        yield f"{self.index}. {element}"


def main():
    # Ustawienia opcji potoku
    pipeline_options = PipelineOptions()
    p = beam.Pipeline(options=pipeline_options)

    # Dane wejściowe w pamięci
    input_data = [
        'Using create transform',
        'to generate in memory data',
        'This is 3rd line',
        'Thanks'
    ]

    lines = (
        p
        # Tworzenie danych wejściowych za pomocą transformacji `Create`
        | 'CreateInput' >> beam.Create(input_data)

        # Dodanie numerów linii (indeksu) za pomocą transformacji `ParDo`
        | 'AddLineNumbers' >> beam.ParDo(AddLineNumbersFn())

        # Filtracja krótkich linii
        | 'FilterShortLines' >> beam.Filter(
            lambda line: len(line) > 20
        )

        # Zapis wyników do pliku tekstowego
        | 'WriteToFiles' >> beam.io.WriteToText(
            'data/processed_text',
            file_name_suffix='.txt',  # Format pliku tekstowego
            num_shards=1,  # Jeden plik
            header='# Processed text data',  # Nagłówek pliku
            footer='# End of file'  # Stopka pliku
        )
    )

    # Uruchomienie potoku
    p.run().wait_until_finish()


if __name__ == '__main__':
    main()

In [20]:
!{('head -n 20 /content/data/processed_text-00000-of-00001.txt')}

# Processed text data
1. Using create transform
2. to generate in memory data
# End of file


##p3

In [23]:
p3 = beam.Pipeline()

lines1 = (p3

           | beam.Create([1,2,3,4,5,6,7,8,9])

           | beam.io.WriteToText('data/outCreate2', file_name_suffix='.txt')
          )
p3.run()

In [24]:
!{('head -n 20 /content/data/outCreate2-00000-of-00001.txt')}

1
2
3
4
5
6
7
8
9


##p4

In [25]:
p4 = beam.Pipeline()

lines = (p4
           | beam.Create([("maths",52),("english",75),("science",82), ("computer",65),("maths",85)])

            | beam.io.WriteToText('data/outCreate3', file_name_suffix='.txt')
          )
p4.run()

In [26]:
!{('head -n 20 /content/data/outCreate3-00000-of-00001.txt')}

('maths', 52)
('english', 75)
('science', 82)
('computer', 65)
('maths', 85)


In [27]:
p5 = beam.Pipeline()

lines = ( p5

       | beam.Create({'row1':[1,2,3,4,5],
                     'row2':[1,2,3,4,5]})

          """Stosuje transformację Map, która w tym przypadku nie zmienia elementów,
          bo przekazywana jest funkcja tożsamościowa (lambda element: element).
          W efekcie każdy element pozostaje taki sam."""
       | beam.Map(lambda element: element)

       | beam.io.WriteToText('data/outCreate4', file_name_suffix='.txt')
  )

p5.run()

In [28]:
!{('head -n 20 /content/data/outCreate4-00000-of-00001.txt')}

('row1', [1, 2, 3, 4, 5])
('row2', [1, 2, 3, 4, 5])
